In [1]:
ref_osm = '/mnt/d/osm2lulc01/osm2lulc01.gdb/osm/osm_evu_forest'

cols_keys = [
    'aeroway', 'amenity', 'barrier', 'building',
    'craft', 'geological', 'historic', 'landuse', 'leisure',
    'man_made', 'military', 'natural', 'office', 'place', 'shop',
    'sport', 'tourism', 'waterway', 'power', 'railway', 'healthcare',
    'highway', 'water', 'landcover', 'industrial', 'leaf_type',
    'material', 'surface', 'leaf_cycle',
]

col_class = 'code_2018'

out_tbl = '/mnt/d/osm2lulc01/tags_statistics2.xlsx'

In [2]:
import pandas as pd

from glass.rd.shp import shp_to_obj
from glass.dtr.mge.pd import merge_df
from glass.wt import obj_to_tbl

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
gdf = shp_to_obj(ref_osm)

In [4]:
classes = gdf[col_class].unique()

gdf['garea'] = gdf.geometry.area

class_df = pd.DataFrame({
    'garea' : gdf.groupby([col_class])['garea'].agg('sum')
}).reset_index()

total_area = class_df.garea.sum()

In [5]:
cls_area = {}

for i, r in class_df.iterrows():
    cls_area[r[col_class]] = r.garea

In [6]:
cls_area

{'14100': 43798519.011843145, '31000': 1089575925.6986272}

In [7]:
class_df

,code_2018,garea
0,14100,4.379852e+07
1,31000,1.089576e+09


In [8]:
# For each key, list values

key_df = {}
for cls in classes:
    cls_df = gdf[gdf[col_class] == cls]

    dfs = []
    
    for _key in cols_keys:
        _df = cls_df[~cls_df[_key].isnull()]
    
        gb = pd.DataFrame({
            'garea' : _df.groupby([_key])['garea'].agg('sum')
        }).reset_index()

        gb['osmtag'] = _key + '=' + gb[_key]

        gb.drop(_key, axis=1, inplace=True)

        gb[f'p_{cls}'] = gb.garea / cls_area[cls] * 100

        gb[f'pt_{cls}'] = gb.garea / total_area * 100

        dfs.append(gb)

    key_df[cls] = merge_df(dfs)

In [9]:
# Join all dataframes of each class

main_df = key_df[classes[0]]

for cls in classes[1:]:
    key_df[cls].rename(columns={'osmtag' : 'osmtag_y'}, inplace=True)

    main_df = main_df.merge(key_df[cls], how='outer', left_on='osmtag', right_on='osmtag_y')

    main_df['osmtag'] = main_df.osmtag.fillna(value=main_df.osmtag_y)

    main_df.drop('osmtag_y', axis=1, inplace=True)

    main_df = main_df.fillna(0)

In [10]:
obj_to_tbl(main_df, out_tbl, sheetsName='statistics')

'/mnt/d/osm2lulc01/tags_statistics2.xlsx'